In [1]:
from IPython import display as D

In [2]:
D.Video('./demo.mkv', width=1080, height=728)

In [3]:
D.Video('./demo_0.3.mp4', width=1080, height=728)

In [14]:
from transformers import AutoTokenizer, AutoProcessor, AutoModelForCTC

import torch

import numpy as np
from pathlib import Path

import librosa

cache_dir = Path('/mnt/d/cache/audio')

model_id = "patrickvonplaten/wav2vec2-base-100h-with-lm"

from sympy import Interval, Union

In [15]:
from tqdm.auto import tqdm
from moviepy import editor

In [3]:
step=3*60
for i, sec in enumerate(range(0,3658,step)):
    editor.ffmpeg_tools.ffmpeg_extract_subclip('./screen_2023-02-27 12-51-04.mkv', sec, sec+step, f'part{i}.mp4')


Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join

In [12]:
# editor.ffmpeg_tools.ffmpeg_extract_subclip('./screen_2023-02-27 12-51-04.mkv', 0, 7*60, 'part1.mp4')

In [3]:
# editor.ffmpeg_tools.ffmpeg_extract_subclip('./screen_2023-02-27 12-51-04.mkv', 12*60, *60, 'part2.mp4')

In [3]:
# editor.ffmpeg_tools.ffmpeg_extract_subclip('./screen_2023-02-27 12-51-04.mkv', 1800, 3658, 'part2.mp4')

In [16]:
model = AutoModelForCTC.from_pretrained(model_id, cache_dir='./')

processor = AutoProcessor.from_pretrained(model_id, cache_dir='./')

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

In [17]:
# video = editor.VideoFileClip(f'./part{p}.mp4')
video = editor.VideoFileClip(f'./screen_2023-02-27 12-51-04.mkv')

In [18]:
audio = video.audio.to_soundarray()
audio

array([[ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       ...,
       [-0.00024414, -0.00024414],
       [-0.00021362, -0.00021362],
       [-0.00018311, -0.00018311]])

In [19]:
audio = audio[:, 0]
audio.shape

(161333235,)

In [20]:
sample_rate=16_000

In [21]:
audio = librosa.resample(audio, orig_sr=44_100, target_sr=16_000)
audio.shape

(58533600,)

In [22]:
int(audio.shape[0]/16_000/chunksize)+1

61

In [22]:
intervals=[]
chunksize = 60 # seconds
for p in tqdm(range(1,int(audio.shape[0]/16_000/chunksize)+2)):
#     if p==3: break
    chunk = audio[(p-1)*sample_rate*chunksize:p*sample_rate*chunksize]
    p_start = (p-1)*chunksize
    p_end = p*chunksize

    chunk = processor(chunk, return_tensors='pt', sampling_rate=16_000).input_values
    chunk.shape

    model.eval()
    with torch.no_grad():
        logits = model(chunk).logits

    output = processor.decode(logits[0].numpy(), output_word_offsets=True)
    pad = 0.3
    time_stamps = [
        [(offset['start_offset']*0.02)-pad+p_start, (offset['end_offset']*0.02)+pad+p_start]
     for offset in output['word_offsets']
    ]
#     if p>1:
#         inter = time_stamps[0]
#         pre = [(intervals[-1]).start, (intervals[-1]).end]
#         interv = union([pre, inter])
#         if len(interv)==1:
#             intervals[-1] = [Interval(*interv) for interv in union([pre, inter])][0]
#         else:
#             intervals.extend(interv)
#         intervals.extend(union(time_stamps[1:]))
#     else:
#         intervals.extend(union(time_stamps))
    intervals.extend((time_stamps))
    
# intervals = union(intervals)
intervals

  0%|          | 0/61 [00:00<?, ?it/s]

[[5.66, 6.62],
 [6.12, 6.84],
 [6.48, 7.3],
 [6.88, 7.72],
 [7.42, 8.040000000000001],
 [7.86, 8.56],
 [8.5, 9.280000000000001],
 [8.959999999999999, 9.66],
 [9.16, 9.82],
 [12.299999999999999, 13.3],
 [12.799999999999999, 13.500000000000002],
 [13.1, 13.920000000000002],
 [13.52, 14.360000000000001],
 [14.059999999999999, 14.680000000000001],
 [14.48, 15.24],
 [14.84, 15.680000000000001],
 [15.82, 16.560000000000002],
 [16.0, 16.64],
 [16.12, 17.12],
 [16.8, 17.64],
 [17.44, 18.14],
 [17.62, 18.92],
 [18.38, 19.18],
 [18.64, 19.34],
 [18.86, 19.6],
 [19.24, 20.18],
 [19.82, 20.62],
 [20.04, 21.240000000000002],
 [21.86, 22.52],
 [21.98, 22.66],
 [22.34, 22.96],
 [22.419999999999998, 23.220000000000002],
 [22.68, 23.34],
 [22.82, 23.700000000000003],
 [23.14, 23.86],
 [23.3, 23.96],
 [23.44, 24.12],
 [23.7, 24.64],
 [24.6, 25.84],
 [26.46, 27.14],
 [26.6, 27.3],
 [26.82, 27.82],
 [27.3, 28.28],
 [28.58, 29.200000000000003],
 [28.74, 29.62],
 [29.1, 30.16],
 [29.98, 30.740000000000002],

In [23]:
keep_range = [
    [2*60+50., 3*60.],
    [55*60+43., 55*60+53.]
]
intervals.extend(keep_range)

In [38]:
keep_range

[[170.0, 180.0], [3343.0, 3353.0]]

In [39]:
intervals

[[5.66, 6.62],
 [6.12, 6.84],
 [6.48, 7.3],
 [6.88, 7.72],
 [7.42, 8.040000000000001],
 [7.86, 8.56],
 [8.5, 9.280000000000001],
 [8.959999999999999, 9.66],
 [9.16, 9.82],
 [12.299999999999999, 13.3],
 [12.799999999999999, 13.500000000000002],
 [13.1, 13.920000000000002],
 [13.52, 14.360000000000001],
 [14.059999999999999, 14.680000000000001],
 [14.48, 15.24],
 [14.84, 15.680000000000001],
 [15.82, 16.560000000000002],
 [16.0, 16.64],
 [16.12, 17.12],
 [16.8, 17.64],
 [17.44, 18.14],
 [17.62, 18.92],
 [18.38, 19.18],
 [18.64, 19.34],
 [18.86, 19.6],
 [19.24, 20.18],
 [19.82, 20.62],
 [20.04, 21.240000000000002],
 [21.86, 22.52],
 [21.98, 22.66],
 [22.34, 22.96],
 [22.419999999999998, 23.220000000000002],
 [22.68, 23.34],
 [22.82, 23.700000000000003],
 [23.14, 23.86],
 [23.3, 23.96],
 [23.44, 24.12],
 [23.7, 24.64],
 [24.6, 25.84],
 [26.46, 27.14],
 [26.6, 27.3],
 [26.82, 27.82],
 [27.3, 28.28],
 [28.58, 29.200000000000003],
 [28.74, 29.62],
 [29.1, 30.16],
 [29.98, 30.740000000000002],

In [40]:
intervals_srt = sorted(intervals, key=lambda x: x[0])

interval_union=[]

for i, l in enumerate(intervals_srt):
    if i==0:
        interval_union.append(l)
        continue
#     for s,e in keep_range:
#         if l[0]>s and l[1]<e:
#             add=False
#     if add==false:
#         continue
        
    prev = interval_union.pop()
    if l[0]<prev[1]:
        interval_union.append([prev[0], l[1]])
    else:
        interval_union.extend([prev, l])
interval_union

[[5.66, 9.82],
 [12.299999999999999, 15.680000000000001],
 [15.82, 21.240000000000002],
 [21.86, 25.84],
 [26.46, 28.28],
 [28.58, 31.34],
 [32.980000000000004, 36.36],
 [36.620000000000005, 39.08],
 [40.120000000000005, 42.54],
 [42.720000000000006, 44.879999999999995],
 [45.260000000000005, 47.26],
 [47.68000000000001, 48.879999999999995],
 [49.52, 51.1],
 [51.220000000000006, 53.339999999999996],
 [53.78, 57.16],
 [57.580000000000005, 59.1],
 [59.300000000000004, 62.68],
 [63.46, 64.9],
 [65.04, 68.14],
 [68.56, 69.74],
 [69.92, 72.06],
 [72.96, 75.14],
 [76.75999999999999, 82.14],
 [82.84, 83.56],
 [83.84, 86.92],
 [87.46000000000001, 91.38],
 [91.86, 94.08],
 [94.5, 97.08],
 [97.42, 101.86],
 [102.1, 106.96000000000001],
 [109.24000000000001, 118.94],
 [119.14, 121.76],
 [121.86, 125.36],
 [125.48, 128.28],
 [128.7, 131.96],
 [132.26, 138.74],
 [139.68, 142.88],
 [143.92, 152.66],
 [153.74, 155.07999999999998],
 [155.44, 157.06],
 [157.64, 159.62],
 [159.88, 160.82],
 [162.0400000

In [42]:
for skeep, ekeep in keep_range:
    for interval in interval_union:
        if interval[0]>=skeep and interval[1]<=ekeep:
            interval_union.remove(interval)
        if interval[0]>=skeep and interval[0]<=ekeep:
            interval[0] = skeep
        if interval[1]>=skeep and interval[1]<=ekeep:
            interval[1] = ekeep

In [44]:
interval_union = sorted(interval_union, key=lambda x: x[0])

In [50]:
interval_union

[[5.66, 9.82],
 [12.299999999999999, 15.680000000000001],
 [15.82, 21.240000000000002],
 [21.86, 25.84],
 [26.46, 28.28],
 [28.58, 31.34],
 [32.980000000000004, 36.36],
 [36.620000000000005, 39.08],
 [40.120000000000005, 42.54],
 [42.720000000000006, 44.879999999999995],
 [45.260000000000005, 47.26],
 [47.68000000000001, 48.879999999999995],
 [49.52, 51.1],
 [51.220000000000006, 53.339999999999996],
 [53.78, 57.16],
 [57.580000000000005, 59.1],
 [59.300000000000004, 62.68],
 [63.46, 64.9],
 [65.04, 68.14],
 [68.56, 69.74],
 [69.92, 72.06],
 [72.96, 75.14],
 [76.75999999999999, 82.14],
 [82.84, 83.56],
 [83.84, 86.92],
 [87.46000000000001, 91.38],
 [91.86, 94.08],
 [94.5, 97.08],
 [97.42, 101.86],
 [102.1, 106.96000000000001],
 [109.24000000000001, 118.94],
 [119.14, 121.76],
 [121.86, 125.36],
 [125.48, 128.28],
 [128.7, 131.96],
 [132.26, 138.74],
 [139.68, 142.88],
 [143.92, 152.66],
 [153.74, 155.07999999999998],
 [155.44, 157.06],
 [157.64, 159.62],
 [159.88, 160.82],
 [162.0400000

In [36]:
interval

[3646.74, 3652.28]

In [51]:
# with open('./time_stamps_keep.txt', 'w') as f:
#     for line in interval_union:
#         f.write(str(line[0]))
#         f.write(',')
#         f.write(str(line[1]))
#         f.write('\n')

In [6]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from moviepy import editor

In [52]:
df = pd.read_csv('./time_stamps_keep.txt', header=None, names=['start', 'end'])
df

,start,end
0,5.66,9.82
1,12.30,15.68
2,15.82,21.24
3,21.86,25.84
4,26.46,28.28
...,...,...
815,3632.32,3634.14
816,3634.72,3638.92
817,3638.92,3640.06
818,3640.16,3646.02


In [53]:
df.head(50)

,start,end
0,5.66,9.82
1,12.30,15.68
2,15.82,21.24
3,21.86,25.84
4,26.46,28.28
5,28.58,31.34
6,32.98,36.36
7,36.62,39.08
8,40.12,42.54
9,42.72,44.88


In [62]:
video = editor.VideoFileClip(f'./IMG_5613.MOV')

lag = 0
lag = 87.35 #seconds 

newVideo=[]
for i, row in tqdm(df.iterrows()):
#     if i==50: break
    start = row['start']+lag
    end = row['end']+lag
    if start<0: start=0
    if end>video.duration: end=video.duration
    newVideo.append(video.subclip(start, end))
newVideo

In [ ]:
final_video = editor.concatenate_videoclips(newVideo)
final_video.write_videofile(f'./face.mp4',
                           audio_fps=48000, audio_codec='aac',
                           codec='libx264',
                           threads=4,
                           )

In [139]:
# newVideo=[]
# for i, interval in enumerate(interval_union):
# #     if i==10: break
#     start = float(interval[0])+lag
#     end = float(interval[1])+lag
#     if start<0: start=0
#     if end>video.duration: end=video.duration
#     newVideo.append(video.subclip(start, end))
# newVideo

In [ ]:
video = editor.VideoFileClip(f'./screen_2023-02-27 12-51-04.mkv')

lag = 0

newVideo=[]
for i, row in tqdm(df.iterrows()):
#     if i==50: break
    start = row['start']+lag
    end = row['end']+lag
    if start<0: start=0
    if end>video.duration: end=video.duration
    newVideo.append(video.subclip(start, end))
newVideo

In [ ]:
final_video = editor.concatenate_videoclips(newVideo)
final_video.write_videofile(f'./screen.mp4',
                           audio_fps=48000, audio_codec='aac',
                           codec='libx264',
                           threads=4,
                           )